In [1]:
import nfl_data_py as nfl
import pandas as pd
import os
import numpy as np
import urllib.request
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnnotationBbox, OffsetImage
import matplotlib.image as mpimg

In [2]:
#clean 2024 data

df_twofour = nfl.import_pbp_data([2024], downcast=True, cache=False, alt_path=None, include_participation=False)

#regular season data only
df_twofour = df_twofour.loc[df_twofour.season_type=='REG']

#remove kick offs, field goals, kneels, etc - only plays with passes, runs, and penalties
df_twofour = df_twofour.loc[(df_twofour.play_type.isin(['no_play','pass','run'])) & (df_twofour.epa.isna()==False)]

#match play call to play type, so QB scrambles still considered pass plays
df_twofour.loc[df_twofour['pass']==1, 'play_type'] = 'pass'
df_twofour.loc[df_twofour.rush==1, 'play_type'] = 'run'

#reset index to skip missing numbers
df_twofour.reset_index(drop=True, inplace=True)

df_twofour.head()

2024 done.
Downcasting floats.


,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,out_of_bounds,home_opening_kickoff,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,xpass,pass_oe
0,61.0,2024_01_ARI_BUF,2024090801,BUF,ARI,REG,1,ARI,away,BUF,...,0.0,0.0,-0.200602,NaN,NaN,NaN,NaN,NaN,0.456761,-45.676102
1,83.0,2024_01_ARI_BUF,2024090801,BUF,ARI,REG,1,ARI,away,BUF,...,0.0,0.0,2.028874,1.345418,9.321221,8.0,0.509778,0.363807,0.576656,42.334431
2,108.0,2024_01_ARI_BUF,2024090801,BUF,ARI,REG,1,ARI,away,BUF,...,0.0,0.0,0.754242,0.882798,5.783560,4.0,0.668478,0.255140,0.426443,57.355690
3,133.0,2024_01_ARI_BUF,2024090801,BUF,ARI,REG,1,ARI,away,BUF,...,0.0,0.0,-0.029602,NaN,NaN,NaN,NaN,NaN,0.225364,-22.536427
4,155.0,2024_01_ARI_BUF,2024090801,BUF,ARI,REG,1,ARI,away,BUF,...,0.0,0.0,-0.247749,NaN,NaN,NaN,NaN,NaN,0.424099,-42.409935


In [14]:

df_all_games = df_twofour[['down', 'passer','posteam', 'epa', 'qb_epa', 'cpoe', 'play_type', 'yards_gained']]

df_weeks_pit = df_all_games.loc[
        (df_twofour['week'].between(7, 12) & (df_twofour['posteam'] == 'PIT'))
         ]


df_weeks_pit.head()


,down,passer,posteam,epa,qb_epa,cpoe,play_type,yards_gained
14344,1.0,None,PIT,-0.119683,-0.119683,NaN,run,4.0
14345,2.0,R.Wilson,PIT,1.346680,1.346680,23.117996,pass,15.0
14346,1.0,None,PIT,1.109385,1.109385,NaN,run,18.0
14347,1.0,R.Wilson,PIT,-0.436227,-0.436227,NaN,pass,1.0
14348,2.0,None,PIT,-0.192108,-0.192108,NaN,run,4.0


In [24]:

df_one_two = df_weeks_pit[df_weeks_pit["down"].isin([1, 2])].groupby("posteam")["epa"].mean().reset_index().rename(columns={"posteam": "Team", "epa": f'EPA Downs One&Two'})
df_one_two

,Team,EPA Downs One&Two
0,PIT,-0.035745


In [19]:
df_three = df_weeks_pit[df_weeks_pit["down"].isin([3])].groupby("posteam")["epa"].mean().reset_index().rename(columns={"posteam": "Team", "epa": "EPA 3rd Down"})

df_three

,Team,EPA 3rd Down
0,PIT,0.075108


In [41]:
#place holder to merge dfs


merged_df = pd.merge(
    df_one_two, 
    df_three, 
    on=['Team'],  # Replace with the actual column names to join on
    #suffixes=('_oneh', '_twoh')  # Optional: differentiate column names
).reset_index(drop=True)



merged_df_style = merged_df.style.hide(axis="index")
merged_df_style

Team,EPA Downs One&Two,EPA 3rd Down
PIT,-0.035745,0.075108


In [28]:
#now check out Russ

df_russ = df_weeks_pit.loc[(df_weeks_pit['passer'] == 'R.Wilson')]

df_russ


,down,passer,posteam,epa,qb_epa,cpoe,play_type,yards_gained
14345,2.0,R.Wilson,PIT,1.346680,1.346680,23.117996,pass,15.0
14347,1.0,R.Wilson,PIT,-0.436227,-0.436227,NaN,pass,1.0
14349,3.0,R.Wilson,PIT,-1.836624,-1.836624,NaN,pass,0.0
14353,1.0,R.Wilson,PIT,-0.220602,-0.220602,12.062335,pass,4.0
14355,3.0,R.Wilson,PIT,-1.788280,-1.788280,-70.478844,pass,0.0
...,...,...,...,...,...,...,...,...
24659,2.0,R.Wilson,PIT,-0.344127,-0.344127,NaN,pass,0.0
24660,3.0,R.Wilson,PIT,0.328518,0.328518,15.461654,pass,11.0
24662,1.0,R.Wilson,PIT,-0.306962,-0.306962,NaN,pass,0.0
24664,2.0,R.Wilson,PIT,0.477841,0.477841,29.483795,pass,9.0


In [35]:
#df_russ_one_two = df_russ[df_russ["down"].isin([1, 2])].groupby("posteam")["qb_epa"].mean()["cpoe"].mean()reset_index().rename(columns={"posteam": "Team", "qb_epa": f'QB EPA Downs One&Two', "cpoe": "CPOE"})

df_russ_one_two = (
    df_russ[df_russ["down"].isin([1, 2])]
    .groupby(["posteam", "passer"])[["qb_epa", "cpoe"]]
    .mean()
    .reset_index()
    .rename(columns={"posteam": "Team", "passer": "QB", "qb_epa": "QB EPA Downs One&Two", "cpoe": "CPOE Down One&Two"})
)

df_russ_one_two

,Team,QB,QB EPA Downs One&Two,CPOE Down One&Two
0,PIT,R.Wilson,0.04556,1.028206


In [34]:
df_russ_three = (
    df_russ[df_russ["down"].isin([3])]
    .groupby(["posteam", "passer"])[["qb_epa", "cpoe"]]
    .mean()
    .reset_index()
    .rename(columns={"posteam": "Team", "passer": "QB", "qb_epa": "QB EPA 3rd Down", "cpoe": "CPOE 3rd Down"})
)

df_russ_three

,Team,QB,QB EPA 3rd Down,CPOE 3Rd Down
0,PIT,R.Wilson,0.222649,15.832744


In [47]:

merged_df_russ = pd.merge(
    df_russ_one_two, 
    df_russ_three, 
    on=['Team', "QB"],  # Replace with the actual column names to join on
    
).reset_index(drop=True)


desired_order = [
    "Team", 
    "QB", 
    "QB EPA Downs One&Two", 
    "QB EPA 3rd Down",
    "CPOE Down One&Two",
    "CPOE 3Rd Down"
]

# Reorder the columns
merged_df_russ = merged_df_russ[desired_order]


merged_df_russ_style = merged_df_russ.style.hide(axis="index")
merged_df_russ_style

Team,QB,QB EPA Downs One&Two,QB EPA 3rd Down,CPOE Down One&Two,CPOE 3Rd Down
PIT,R.Wilson,0.045560,0.222649,1.028206,15.832744
